In [1]:
# Dependencies
from bokeh.core.validation import silence
from bokeh.core.validation.warnings import FIXED_SIZING_MODE
# import json
# import matplotlib.pyplot as plt
# import numpy as np
import pandas as pd
import hvplot.pandas
# from pprint import pprint
import random
import requests
# import scipy.stats as st

# Import API Keys
from config import geoapify_key

# Silence hvplot warning
silence(FIXED_SIZING_MODE, True)

{Warning(code=1005, name='FIXED_SIZING_MODE', description="'fixed' sizing mode requires width and height to be set")}

In [2]:
crime_data_path = "../data/world_crime_index.csv"

crime_data = pd.read_csv(crime_data_path)

crime_data.head()

,Rank,City,Crime Index,Safety Index
0,1,"Caracas, Venezuela",83.98,16.02
1,2,"Pretoria, South Africa",81.98,18.02
2,3,"Celaya, Mexico",81.80,18.20
3,4,"San Pedro Sula, Honduras",80.87,19.13
4,5,"Port Moresby, Papua New Guinea",80.71,19.29


In [3]:
cities = []
crime_index = []
# random_nums = random.sample(range(0, len(crime_data)), 1)

counter = 0
for row in crime_data["Rank"]:
    cities.append(crime_data["City"][counter])
    crime_index.append(crime_data["Crime Index"][counter])
    counter += 1

# pprint(cities)
# pprint(crime_index)

In [4]:
geo_data = []
for city in cities:
    target_url = f"https://api.geoapify.com/v1/geocode/search?text={city}&format=json&apiKey={geoapify_key}"
    req = requests.get(target_url).json()
    geo_data.append(req)

# print(len(geo_data))
# print(json.dumps(geo_data, indent=4, sort_keys=True))

In [5]:
lon = []
lat = []
city = []
country = []
city_data = {
    "city": city,
    "country": country,
    "lon": lon,
    "lat": lat,
    "crime_index": crime_index,
}

counter = 0
for data in geo_data:
    result = geo_data[counter]["results"]
    city_index = 0
    for obj in result:
        if "city" in obj:
            city.append(result[city_index]["city"])
            break
        else:
            city_index += 1
    country_index = 0
    for obj in result:
        if "country" in obj:
            country.append(result[country_index]["country"])
            break
        else:
            country_index += 1
    lon_index = 0
    for obj in result:
        if "lon" in obj:
            lon.append(result[lon_index]["lon"])
            break
        else:
            lon_index += 1
    lat_index = 0
    for obj in result:
        if "lat" in obj:
            lat.append(result[lat_index]["lat"])
            break
        else:
            lat_index += 1
    counter += 1

In [6]:
city_data_df = pd.DataFrame(city_data)
city_data_df.to_csv("../data/city_data.csv", index=False)

In [7]:
crime_index_map_plot = city_data_df.hvplot.points(
    "lon",
    "lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    s="crime_index",
    c="crime_index",
    scale=2,
    alpha=0.9,
    hover=True,
    hover_cols=["lon", "lat", "city", "country", "crime_index"],
)

hvplot.save(crime_index_map_plot, "../data/crime_index_map.html")

crime_index_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (crime_index,city,country)